In [10]:
import os


In [2]:
%pwd

'c:\\Users\\Korisnik\\Desktop\\ml_projects\\ds_resume_light_project\\ds_resume_light_project\\research'

In [3]:
os.chdir("../")

%pwd

'c:\\Users\\Korisnik\\Desktop\\ml_projects\\ds_resume_light_project\\ds_resume_light_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from src.ds_resume_light_project.constants import *
from src.ds_resume_light_project.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
      
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [7]:
import os
from urllib import request
from src.ds_resume_light_project import logger
import zipfile

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"File : {filename} downloaded successfully with following info: \n{headers}")
        else:
            logger.info(f"File already exists of path: {self.config.local_data_file}")

    def extract_zip_file(self) -> None:
        """zip_file_path: str
            Extracts the zip file into the data directory
            Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted successfully at location: {unzip_path}")

In [12]:
try:
    config = ConfigurationManager()
    get_data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = get_data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-24 20:01:54,592: INFO: common]: YAML file: config\config.yaml loaded successfully
[2026-01-24 20:01:54,595: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-24 20:01:54,597: INFO: common]: YAML file: schema.yaml loaded successfully
[2026-01-24 20:01:54,598: INFO: common]: Directory created at: artifacts
[2026-01-24 20:01:54,599: INFO: common]: Directory created at: artifacts/data_ingestion
[2026-01-24 20:01:55,597: INFO: 3219601873]: File : artifacts\data_ingestion\data.zip downloaded successfully with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4664:113F0E:220EFAE:26676F7:69751735
Accept-Ran